In [27]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize the WebDriver
print("Initializing Chrome WebDriver...")
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

# Rating conversion dictionaries for each agency
# Higher number = better credit rating

# S&P ratings conversion (AAA = 22, D = 1)
sp_ratings = {
    'AAA': 22, 'AA+': 21, 'AA': 20, 'AA-': 19,
    'A+': 18, 'A': 17, 'A-': 16,
    'BBB+': 15, 'BBB': 14, 'BBB-': 13,
    'BB+': 12, 'BB': 11, 'BB-': 10,
    'B+': 9, 'B': 8, 'B-': 7,
    'CCC+': 6, 'CCC': 5, 'CCC-': 4,
    'CC': 3, 'C': 2, 'D': 1,
    # Note: SD and RD are converted to 'D' in extract_base_rating()
}

# Moody's ratings conversion (Aaa = 22, C = 1)
moodys_ratings = {
    'Aaa': 22, 'Aa1': 21, 'Aa2': 20, 'Aa3': 19,
    'A1': 18, 'A2': 17, 'A3': 16,
    'Baa1': 15, 'Baa2': 14, 'Baa3': 13,
    'Ba1': 12, 'Ba2': 11, 'Ba3': 10,
    'B1': 9, 'B2': 8, 'B3': 7,
    'Caa1': 6, 'Caa2': 5, 'Caa3': 4,
    'Ca': 3, 'C': 1
}

# Fitch ratings conversion (AAA = 22, D = 1)
fitch_ratings = {
    'AAA': 22, 'AA+': 21, 'AA': 20, 'AA-': 19,
    'A+': 18, 'A': 17, 'A-': 16,
    'BBB+': 15, 'BBB': 14, 'BBB-': 13,
    'BB+': 12, 'BB': 11, 'BB-': 10,
    'B+': 9, 'B': 8, 'B-': 7,
    'CCC+': 6, 'CCC': 5, 'CCC-': 4,
    'CC': 3, 'C': 2, 'D': 1
    # Note: RD is converted to 'D' in extract_base_rating()
}

# Counter for debug logging
debug_count = 0

# Helper function to extract base rating (without outlook or watch indicators)
def extract_base_rating(rating_text):
    # Strip upgrade/downgrade indicators (but don't remove them completely from the original data)
    clean_rating = rating_text.replace('[upgrade]', '').replace('[downgrade]', '').strip()
    
    # Handle special cases
    if clean_rating in ['N/A', '-', '', 'NR']:
        return 'N/A'
    if clean_rating in ['SD', 'RD']:
        return 'D'  # Treat Selective Default and Restricted Default as Default
    
    # Extract the rating code using regex
    # This will extract the letter-based rating without outlook indicators
    match = re.search(r'([A-Za-z]{1,3}[\+\-]?[123]?)', clean_rating)
    if match:
        return match.group(1)
    return clean_rating

# Convert rating to numeric value
def convert_rating_to_numeric(rating_text, rating_dict):
    global debug_count
    
    if not rating_text:
        return None
    
    # Debug print only for first few conversions
    debug = debug_count < 20
    
    if debug:
        # Debug: Print the exact rating text received
        print(f"Converting rating: '{rating_text}'")
    
    # Check for upgrade/downgrade indicators - ensure it's a string first
    upgrade_modifier = 0
    if isinstance(rating_text, str):
        if '[upgrade]' in rating_text:
            upgrade_modifier = 1/3  # Add 1/3 point for upgrade
            if debug:
                print(f"  Found upgrade indicator, adding +1/3")
        elif '[downgrade]' in rating_text:
            upgrade_modifier = -1/3  # Subtract 1/3 point for downgrade
            if debug:
                print(f"  Found downgrade indicator, subtracting -1/3")
        
    # Get base rating without indicators
    base_rating = extract_base_rating(rating_text)
    if debug:
        print(f"  Base rating: '{base_rating}'")
    
    # Handle N/A and similar cases
    if base_rating == 'N/A':
        if debug:
            print(f"  Rating is N/A, returning None")
        return None
        
    # Get the base numeric value
    base_numeric = rating_dict.get(base_rating, None)
    if debug:
        print(f"  Base numeric value: {base_numeric}")
    
    # Return None if rating not found
    if base_numeric is None:
        if debug:
            print(f"  Rating not found in dictionary, returning None")
        return None
        
    # Apply the upgrade/downgrade modifier
    final_value = base_numeric + upgrade_modifier
    if debug:
        print(f"  Final value with modifier: {final_value}")
    
    # Increment debug counter
    debug_count += 1
        
    return final_value

# First script function: Scrape bond spreads - CORRECTED VERSION
# First script function: Scrape bond spreads - HARDCODED VERSION
# First script function: Scrape bond spreads - FIXED INDICES
def scrape_bond_spreads():
    print("\n=========================================")
    print("PART 1: SCRAPING BOND SPREADS DATA")
    print("=========================================\n")
    
    try:
        # Navigate to the website
        url = "https://www.worldgovernmentbonds.com/spread-historical-data/"
        print(f"Navigating to {url}...")
        driver.get(url)
        
        # Add a longer wait to ensure all elements load
        print("Waiting for page to fully load...")
        time.sleep(5)  # Wait 5 seconds for any dynamic content
        
        # Find all tables directly
        tables = driver.find_elements(By.TAG_NAME, "table")
        print(f"Found {len(tables)} tables on the page")
        
        if not tables:
            print("No tables found, cannot proceed")
            return False
            
        # Get the largest table
        target_table = max(tables, key=lambda t: len(t.find_elements(By.TAG_NAME, "tr")))
        all_rows = target_table.find_elements(By.TAG_NAME, "tr")
        
        # Print the first few rows to confirm structure
        print("\nTABLE STRUCTURE:")
        for i, row in enumerate(all_rows[:5]):
            headers = row.find_elements(By.TAG_NAME, "th")
            cells = row.find_elements(By.TAG_NAME, "td")
            
            if headers:
                header_texts = [h.text.strip() for h in headers]
                print(f"Row {i} (HEADER): {header_texts}")
            
            if cells:
                cell_texts = [c.text.strip() for c in cells]
                print(f"Row {i} (DATA): {cell_texts}")
        
        # CORRECT INDICES based on debug output
        country_column_index = 1  # "Country" is at index 1
        usa_column_index = 4      # "Usa" is at index 4
        
        print(f"Using indices: Country={country_column_index}, USA={usa_column_index}")
        
        # Start from row 2 (index 2) to skip headers
        data_rows = []
        data_start_row = 2
        
        for i, row in enumerate(all_rows[data_start_row:]):
            cells = row.find_elements(By.TAG_NAME, "td")
            
            # Debug first few rows
            if i < 5:
                print(f"Data row {i+data_start_row}, Cell count: {len(cells)}")
                if len(cells) > max(country_column_index, usa_column_index):
                    country = cells[country_column_index].text.strip()
                    usa_value = cells[usa_column_index].text.strip()
                    print(f"  Country: '{country}', USA value: '{usa_value}'")
            
            # Skip rows with insufficient cells
            if len(cells) <= max(country_column_index, usa_column_index):
                continue
                
            country = cells[country_column_index].text.strip()
            usa_value = cells[usa_column_index].text.strip()
            
            # Only add rows with both country and USA value
            if country and usa_value:
                data_rows.append([country, usa_value])
        
        print(f"\nExtracted {len(data_rows)} rows with country and USA spread values")
        
        # Print the first few rows to confirm
        print("\nFirst 5 extracted rows:")
        for i, row in enumerate(data_rows[:5]):
            print(f"{i+1}. Country: '{row[0]}', USA value: '{row[1]}'")
        
        # Create DataFrame
        if data_rows:
            df = pd.DataFrame(data_rows, columns=["Country", "Spread"])
            
            # Remove any bp or % symbols and convert to numeric
            df["Spread"] = df["Spread"].str.replace(',', '').str.replace('%', '').str.replace('bp', '').str.strip()
            df["Spread"] = pd.to_numeric(df["Spread"], errors='coerce')
            
            # Remove rows with empty country or NaN spread
            df = df[df['Country'].str.strip() != '']
            df = df.dropna(subset=['Spread'])
            
            # Save to CSV
            output_file = "government_bond_spreads.csv"
            df.to_csv(output_file, index=False)
            print(f"\nData saved to {output_file}")
            
            # Print preview
            print("\nPreview of the bond spreads data (first 5 rows):")
            print(df.head().to_string())
            
            return True
            
        else:
            print("No data rows extracted")
            return False

    except Exception as e:
        print(f"An error occurred during bond spreads scraping: {e}")
        import traceback
        traceback.print_exc()
        
        return False

# Second script function: Scrape credit ratings
def scrape_credit_ratings():
    print("\n=========================================")
    print("PART 2: SCRAPING CREDIT RATINGS DATA")
    print("=========================================\n")
    
    try:
        # Navigate to the website
        url = "https://www.worldgovernmentbonds.com/world-credit-ratings/"
        print(f"Navigating to {url}...")
        driver.get(url)
        
        # Add a longer wait to ensure all elements load
        print("Waiting for page to fully load...")
        time.sleep(5)  # Wait 5 seconds for any dynamic content
        
        # Wait for tables to load
        print("Waiting for tables to load...")
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, "table"))
        )
        
        # Find all tables
        tables = driver.find_elements(By.TAG_NAME, "table")
        print(f"Found {len(tables)} tables on the page")
        
        # Use the largest table
        if tables:
            target_table = max(tables, key=lambda t: len(t.find_elements(By.TAG_NAME, "tr")))
            
            # Extract table structure
            rows = target_table.find_elements(By.TAG_NAME, "tr")
            print(f"Found {len(rows)} rows in the target table")
            
            # Extract headers
            header_row = rows[0]
            headers = [header.text.strip() for header in header_row.find_elements(By.TAG_NAME, "th")]
            print(f"Headers: {headers}")
            
            # Find the indexes of the S&P, Moody's, and Fitch columns
            rating_column_indices = {}
            dbrs_index = None
            
            for i, header in enumerate(headers):
                header_lower = header.lower()
                if "s&p" in header_lower:
                    rating_column_indices["S&P"] = i
                    print(f"Found S&P column: {header} at index {i}")
                elif "moody" in header_lower:
                    rating_column_indices["Moody's"] = i
                    print(f"Found Moody's column: {header} at index {i}")
                elif "fitch" in header_lower:
                    rating_column_indices["Fitch"] = i
                    print(f"Found Fitch column: {header} at index {i}")
                elif "dbrs" in header_lower:
                    dbrs_index = i
                    print(f"Found DBRS column: {header} at index {i} - will be removed")
            
            # Create specific selectors for red and teal indicators
            red_selector = "i.w3-text-red.fa.fa-circle.w3-tiny"
            teal_selector = "i.w3-text-teal.fa.fa-circle.w3-tiny"
            
            # More general fallback selectors
            red_fallback_selector = "i[class*='w3-text-red'][class*='fa-circle']"
            teal_fallback_selector = "i[class*='w3-text-teal'][class*='fa-circle']"
            
            # Extract data rows with indicators
            data_rows = []
            for row in rows[1:]:  # Skip header row
                cells = row.find_elements(By.TAG_NAME, "td")
                if cells and len(cells) > 0:
                    row_data = []
                    
                    for i, cell in enumerate(cells):
                        # Skip the DBRS column
                        if i == dbrs_index:
                            continue
                            
                        # Only process cells that exist in the row
                        if i < len(cells):
                            cell_text = cell.text.strip()
                            
                            # If this is a rating column, check for indicators
                            if i in rating_column_indices.values():
                                # Look for red indicator
                                red_icons = cell.find_elements(By.CSS_SELECTOR, red_selector)
                                if not red_icons:  # Try fallback if exact match fails
                                    red_icons = cell.find_elements(By.CSS_SELECTOR, red_fallback_selector)
                                
                                # Look for teal indicator
                                teal_icons = cell.find_elements(By.CSS_SELECTOR, teal_selector)
                                if not teal_icons:  # Try fallback if exact match fails
                                    teal_icons = cell.find_elements(By.CSS_SELECTOR, teal_fallback_selector)
                                
                                # Add indicator to cell text
                                if red_icons:
                                    cell_text = f"{cell_text} [downgrade]"
                                    print(f"Found downgrade indicator for cell: {cell_text}")
                                elif teal_icons:
                                    cell_text = f"{cell_text} [upgrade]"
                                    print(f"Found upgrade indicator for cell: {cell_text}")
                            
                            row_data.append(cell_text)
                    
                    # Only add rows with data
                    if row_data:
                        data_rows.append(row_data)
            
            # Create new headers without DBRS
            new_headers = []
            for i, header in enumerate(headers):
                if i != dbrs_index:
                    new_headers.append(header)
            
            # Create DataFrame
            if new_headers and data_rows:
                # Make sure the dimensions match
                if len(new_headers) != len(data_rows[0]):
                    print(f"Warning: Headers count ({len(new_headers)}) doesn't match column count in first row ({len(data_rows[0])})")
                    
                    if len(new_headers) > len(data_rows[0]):
                        # Truncate headers to match data
                        new_headers = new_headers[:len(data_rows[0])]
                    else:
                        # Add generic column names if needed
                        while len(new_headers) < len(data_rows[0]):
                            new_headers.append(f"Column_{len(new_headers)+1}")
                
                # Create DataFrame with headers and data
                df = pd.DataFrame(data_rows, columns=new_headers)
                
                # Add numeric conversion columns for each rating agency
                
                # Map the original column names to their rating dictionaries
                rating_mapping = {}
                
                # Find exact column names
                for agency, abbr in [("s&p", "S&P"), ("moody", "Moody's"), ("fitch", "Fitch")]:
                    found = False
                    for header in new_headers:
                        if agency in header.lower():
                            # Fix variable name for Moody's ratings
                            if agency == "moody":
                                rating_dict_name = "moodys_ratings"
                            else:
                                rating_dict_name = f"{agency.replace('&', '')}_ratings"
                            
                            rating_mapping[header] = (f"{abbr}_Numeric", globals()[rating_dict_name])
                            print(f"Mapped '{header}' to '{abbr}_Numeric' using {rating_dict_name}")
                            found = True
                            break
                    if not found:
                        print(f"Warning: Could not find column for {abbr}")
                
                # Add numeric conversion columns
                for col_name, (numeric_col_name, rating_dict) in rating_mapping.items():
                    print(f"\nConverting ratings for column: {col_name}")
                    # Print a few sample values from the original column
                    print("Sample values from original column:")
                    for i, val in enumerate(df[col_name].head(10)):
                        print(f"  {i+1}. '{val}'")
                    
                    # Convert to numeric
                    df[numeric_col_name] = df[col_name].apply(
                        lambda x: convert_rating_to_numeric(x, rating_dict)
                    )
                    
                    # Print result preview
                    print(f"Preview of converted values in {numeric_col_name}:")
                    for i, (orig, num) in enumerate(zip(df[col_name].head(10), df[numeric_col_name].head(10))):
                        print(f"  {i+1}. '{orig}' → {num}")
                
                # Add average rating column (ignoring None values)
                df['Average_Rating'] = df[['S&P_Numeric', 'Moody\'s_Numeric', 'Fitch_Numeric']].mean(axis=1, skipna=True).round(2)
                
                # Add count of available ratings column
                df['Ratings_Count'] = df[['S&P_Numeric', 'Moody\'s_Numeric', 'Fitch_Numeric']].count(axis=1)
                
                print("Saving credit ratings data to CSV...")
                output_file = "world_credit_ratings_with_numeric.csv"
                df.to_csv(output_file, index=False)
                print(f"Data saved to {output_file}")
                
                # Display count of upgrade/downgrade indicators
                indicator_stats = {
                    'upgrade': 0,
                    'downgrade': 0,
                    'no_indicator': 0
                }
                
                # Check all rating columns for indicator counts
                for header in new_headers:
                    if any(agency in header.lower() for agency in ['s&p', 'moody', 'fitch']):
                        for val in df[header]:
                            if isinstance(val, str):
                                if '[upgrade]' in val:
                                    indicator_stats['upgrade'] += 1
                                elif '[downgrade]' in val:
                                    indicator_stats['downgrade'] += 1
                                else:
                                    indicator_stats['no_indicator'] += 1
                
                print("\nIndicator Statistics:")
                print(f"Ratings with [upgrade] indicator: {indicator_stats['upgrade']}")
                print(f"Ratings with [downgrade] indicator: {indicator_stats['downgrade']}")
                print(f"Ratings without indicators: {indicator_stats['no_indicator']}")
                
                # Print preview showing both original and numeric columns
                print("\nPreview of the credit ratings data (showing first few countries with ratings):")
                
                try:
                    # Create a preview with any available columns
                    available_columns = df.columns.tolist()
                    
                    # Try to find a good first column that might contain country names
                    first_column = available_columns[0]  # Default to first column
                    for col in available_columns:
                        if col.lower() == 'country':
                            first_column = col
                            print(f"Using column '{col}' as country column")
                            break
                    
                    # Try to find agency columns
                    preview_columns = [first_column]
                    for agency in ['S&P', 'Moody\'s', 'Fitch']:
                        # Find original rating column
                        agency_cols = [col for col in available_columns if agency.lower() in col.lower() and 'numeric' not in col.lower()]
                        # Find numeric rating column
                        numeric_cols = [col for col in available_columns if f"{agency}_Numeric".lower() in col.lower()]
                        
                        if agency_cols:
                            preview_columns.append(agency_cols[0])
                        if numeric_cols:
                            preview_columns.append(numeric_cols[0])
                    
                    # Print preview
                    print(df[preview_columns].head(5).to_string())
                    
                except Exception as e:
                    print(f"Could not display preview due to error: {e}")
                    print("Displaying first few rows with all columns instead:")
                    print(df.head(3))
                
                return True
                
            else:
                print("Could not extract proper credit ratings data")
                return False
        else:
            print("No tables found on the credit ratings page")
            return False

    except Exception as e:
        print(f"An error occurred during credit ratings scraping: {e}")
        
        # Save the page source in case of error
        with open("credit_ratings_error_page.html", "w", encoding="utf-8") as f:
            f.write(driver.page_source)
        print("Saved page source to 'credit_ratings_error_page.html' for debugging")
        return False

# Main execution
try:
    print("Starting combined scraping of bond spreads and credit ratings...")
    
    # First, scrape bond spreads
    spreads_successful = scrape_bond_spreads()
    
    # Then, scrape credit ratings
    ratings_successful = scrape_credit_ratings()
    
    # Summary of what was accomplished
    print("\n=========================================")
    print("SCRAPING SUMMARY")
    print("=========================================")
    
    if spreads_successful:
        print("✅ Bond spreads data successfully scraped and saved to 'government_bond_spreads.csv'")
    else:
        print("❌ Failed to scrape bond spreads data")
        
    if ratings_successful:
        print("✅ Credit ratings data successfully scraped and saved to 'world_credit_ratings_with_numeric.csv'")
    else:
        print("❌ Failed to scrape credit ratings data")
    
    if spreads_successful and ratings_successful:
        print("\nBoth datasets were successfully scraped! 🎉")
    elif spreads_successful or ratings_successful:
        print("\nPartial success: only one dataset was scraped successfully.")
    else:
        print("\nFailed to scrape both datasets.")

except Exception as e:
    print(f"An unexpected error occurred in the main execution: {e}")

finally:
    # Close the driver
    driver.quit()
    print("\nWebDriver closed")

Initializing Chrome WebDriver...
Starting combined scraping of bond spreads and credit ratings...

PART 1: SCRAPING BOND SPREADS DATA

Navigating to https://www.worldgovernmentbonds.com/spread-historical-data/...
Waiting for page to fully load...
Found 1 tables on the page

TABLE STRUCTURE:
Row 0 (HEADER): ['', '', '', 'Spread vs']
Row 1 (HEADER): ['', 'Country', '10Y Yield▴', 'Ger', 'Usa', 'Chi', 'Aus']
Row 2 (DATA): ['', 'Switzerland', '0.305%', '-224.7 bp', '-407.6 bp', '-132.5 bp', '-398.8 bp']
Row 3 (DATA): ['', 'Japan', '1.356%', '-119.6 bp', '-302.5 bp', '-27.4 bp', '-293.7 bp']
Row 4 (DATA): ['', 'Taiwan', '1.530%', '-102.2 bp', '-285.1 bp', '-10.0 bp', '-276.3 bp']
Using indices: Country=1, USA=4
Data row 2, Cell count: 7
  Country: 'Switzerland', USA value: '-407.6 bp'
Data row 3, Cell count: 7
  Country: 'Japan', USA value: '-302.5 bp'
Data row 4, Cell count: 7
  Country: 'Taiwan', USA value: '-285.1 bp'
Data row 5, Cell count: 7
  Country: 'China', USA value: '-275.1 bp'
Da

In [28]:
import pandas as pd
import re

def clean_country_name(name):
    """
    Clean the country name by removing any asterisks and parentheses,
    and stripping whitespace.
    """
    if not isinstance(name, str):
        return name
    
    # Remove (*) and any other parenthetical expressions
    name = re.sub(r'\s*\([^)]*\)\s*', '', name)
    
    # Strip any remaining whitespace
    return name.strip()

# Load the datasets
try:
    print("Loading the CSV files...")
    
    # Load bond spreads data
    spread_df = pd.read_csv("government_bond_spreads.csv")
    print(f"Loaded bond spreads data with {len(spread_df)} rows")
    print(f"Bond spreads columns: {spread_df.columns.tolist()}")
    print("\nSample bond spreads data (first 5 rows):")
    print(spread_df.head().to_string())
    
    # Load credit ratings data
    ratings_df = pd.read_csv("world_credit_ratings_with_numeric.csv")
    print(f"\nLoaded credit ratings data with {len(ratings_df)} rows")
    print(f"Credit ratings columns: {ratings_df.columns.tolist()}")
    print("\nSample credit ratings data (first 5 rows):")
    print(ratings_df.head().to_string())
    
    # FIX: Correctly identify the country column in the ratings dataframe
    # Looking at the sample output, it seems 'Country▴' is the correct column name
    country_column = 'Country▴'
    print(f"\nUsing '{country_column}' as the country column for ratings data.")
    
    # Clean country names in both dataframes
    print("\nCleaning country names...")
    
    # Clean country names in spread data
    spread_df['Country_Clean'] = spread_df['Country'].apply(clean_country_name)
    print("\nExample of country name cleaning in spreads data:")
    for idx, (orig, cleaned) in enumerate(zip(spread_df['Country'].head(10), spread_df['Country_Clean'].head(10))):
        if orig != cleaned:
            print(f"  {orig} -> {cleaned}")
    
    # Clean country names in ratings data
    ratings_df['Country_Clean'] = ratings_df[country_column].apply(clean_country_name)
    print("\nExample of country name cleaning in ratings data:")
    for idx, (orig, cleaned) in enumerate(zip(ratings_df[country_column].head(10), ratings_df['Country_Clean'].head(10))):
        if orig != cleaned:
            print(f"  {orig} -> {cleaned}")
    
    # Merge datasets on the cleaned country names
    print("\nMerging datasets on cleaned country names...")
    merged_df = pd.merge(
        spread_df, 
        ratings_df,
        left_on='Country_Clean',
        right_on='Country_Clean',
        how='inner'
    )
    
    # Count the matches
    print(f"\nSuccessfully merged {len(merged_df)} countries")
    print(f"Original spread_df had {len(spread_df)} countries")
    print(f"Original ratings_df had {len(ratings_df)} countries")
    
    # Count how many countries didn't match
    unmatched_spreads = set(spread_df['Country_Clean']) - set(ratings_df['Country_Clean'])
    unmatched_ratings = set(ratings_df['Country_Clean']) - set(spread_df['Country_Clean'])
    
    print(f"\nCountries in spread data but not in ratings data: {len(unmatched_spreads)}")
    if unmatched_spreads:
        print("Sample unmatched spreads countries (up to 10):")
        for country in list(unmatched_spreads)[:10]:
            print(f"  {country}")
    
    print(f"\nCountries in ratings data but not in spread data: {len(unmatched_ratings)}")
    if unmatched_ratings:
        print("Sample unmatched ratings countries (up to 10):")
        for country in list(unmatched_ratings)[:10]:
            print(f"  {country}")
    
    # Reorganize columns in the final dataset
    # Keep only the original Country column from spread_df and drop the cleaning columns
    cols_to_keep = ['Country', 'Spread']  # Start with these columns from spread_df
    
    # Add all columns from ratings_df except the country column and Country_Clean
    for col in ratings_df.columns:
        if col != country_column and col != 'Country_Clean':
            cols_to_keep.append(col)
    
    # Make sure all columns in cols_to_keep exist in the merged dataframe
    cols_to_keep = [col for col in cols_to_keep if col in merged_df.columns]
    
    # Reorder columns
    merged_df = merged_df[cols_to_keep]
    
    # Sort by average rating (if available) or alphabetically by country
    if 'Average_Rating' in merged_df.columns:
        merged_df = merged_df.sort_values(by='Average_Rating', ascending=False)
        print("\nSorted data by Average_Rating (descending)")
    else:
        merged_df = merged_df.sort_values(by='Country')
        print("\nSorted data alphabetically by Country")
    
    # Save the merged dataset
    output_file = "credit_ratings_and_spreads.csv"
    merged_df.to_csv(output_file, index=False)
    print(f"\nMerged data saved to {output_file}")
    
    # Print a preview of the final merged data
    print("\nPreview of the merged data (first 5 rows):")
    print(merged_df.head().to_string())
    
    # Print column stats of the final merged data
    print(f"\nFinal dataset has {len(merged_df)} rows and {len(merged_df.columns)} columns")
    print(f"Columns: {merged_df.columns.tolist()}")
    
except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Make sure both CSV files are in the current directory.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    import traceback
    traceback.print_exc()  # Print full traceback for debugging

Loading the CSV files...
Loaded bond spreads data with 70 rows
Bond spreads columns: ['Country', 'Spread']

Sample bond spreads data (first 5 rows):
       Country  Spread
0  Switzerland  -407.6
1        Japan  -302.5
2       Taiwan  -285.1
3        China  -275.1
4      Denmark  -205.6

Loaded credit ratings data with 74 rows
Credit ratings columns: ['Unnamed: 0', 'Country▴', 'S&P', "Moody's", 'Fitch', 'S&P_Numeric', "Moody's_Numeric", 'Fitch_Numeric', 'Average_Rating', 'Ratings_Count']

Sample credit ratings data (first 5 rows):
   Unnamed: 0    Country▴             S&P         Moody's            Fitch  S&P_Numeric  Moody's_Numeric  Fitch_Numeric  Average_Rating  Ratings_Count
0         NaN   Argentina             CCC  Caa3 [upgrade]              CCC     5.000000         4.333333       5.000000            4.78              3
1         NaN   Australia             AAA             Aaa              AAA    22.000000        22.000000      22.000000           22.00              3
2         N

In [29]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Bond spreads vs. sovereign credit ratings

• 12×12-inch figure, double fonts
• Investment-grade vs. speculative shading
• Vertical x = 0 reference line
• Vertical ties between multiple agency ratings per country
• PNG flags (any size) or 2-letter ISO fallback
• Correlation line with coefficient displayed
"""

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from pathlib import Path
import sys

# ---------------------------------------------------------------- paths -------
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:                    # e.g. Jupyter
    BASE_DIR = Path.cwd()

CSV_FILE = BASE_DIR / "credit_ratings_and_spreads.csv"
ICON_DIR = BASE_DIR / "flag_icons"        # us.png, gb.png, au.png, …

# ---------------------------------------------------------------- ISO map ----
COUNTRY_TO_CODE =  {
    "Switzerland": "ch", "Singapore": "sg", "Norway": "no", "Netherlands": "nl",
    "Germany": "de", "Australia": "au", "Sweden": "se", "Denmark": "dk",
    "Canada": "ca", "New Zealand": "nz", "United States": "us", "USA": "us",
    "Finland": "fi", "Austria": "at", "Qatar": "qa", "Taiwan": "tw",
    "Ireland": "ie", "South Korea": "kr", "Korea, South": "kr", "Hong Kong": "hk",
    "United Kingdom": "gb", "UK": "gb", "Belgium": "be", "Czech Republic": "cz",
    "Czechia": "cz", "France": "fr", "Iceland": "is", "Slovenia": "si",
    "Japan": "jp", "China": "cn", "Lithuania": "lt", "Malta": "mt",
    "Chile": "cl", "Portugal": "pt", "Slovakia": "sk", "Poland": "pl",
    "Spain": "es", "Croatia": "hr", "Cyprus": "cy", "Israel": "il",
    "Malaysia": "my", "Botswana": "bw", "Bulgaria": "bg", "Philippines": "ph",
    "Italy": "it", "Indonesia": "id", "Peru": "pe", "Kazakhstan": "kz",
    "Mexico": "mx", "Hungary": "hu", "Greece": "gr", "India": "in",
    "Mauritius": "mu", "Romania": "ro", "Colombia": "co", "Serbia": "rs",
    "Morocco": "ma", "Vietnam": "vn", "Brazil": "br", "South Africa": "za",
    "Jordan": "jo", "Namibia": "na", "Turkey": "tr", "Bangladesh": "bd",
    "Bahrain": "bh", "Uganda": "ug", "Nigeria": "ng", "Egypt": "eg",
    "Kenya": "ke", "Pakistan": "pk", "Sri Lanka": "lk", "Zambia": "zm",
    "Ukraine": "ua", "Russia": "ru"
}

# rating number → letter
RATING_LABEL = {22:"AAA",21:"AA+",20:"AA",19:"AA-",18:"A+",17:"A",16:"A-",
                15:"BBB+",14:"BBB",13:"BBB-",12:"BB+",11:"BB",10:"BB-",
                 9:"B+",  8:"B",  7:"B-", 6:"CCC+",5:"CCC",4:"CCC-",
                 3:"CC",  2:"C",  1:"D"}

# ---------------------------------------------------------------- helpers ----
def load_icon(code: str):
    png = ICON_DIR / f"{code}.png"
    return plt.imread(png) if png.exists() else None

# ---------------------------------------------------------------- main plot --
def create_plot(outfile="bond_spreads_vs_ratings.png",
                jitter_x=5, label_dx=8, flag_height=24):

    # ---------- style ------------------------------------------------------
    mpl.rcParams.update({
        "figure.figsize": (12, 12),
        "axes.titlesize": 40, "axes.labelsize": 32,
        "xtick.labelsize": 24, "ytick.labelsize": 24,
        "legend.fontsize": 24,
        "font.family": ["Arial", "Helvetica", "DejaVu Sans", "sans-serif"],
    })
    palette = sns.color_palette("bright", 3)
    rng      = np.random.default_rng()

    # ---------- data -------------------------------------------------------
    df = pd.read_csv(CSV_FILE)
    df["Spread_Numeric"] = (df["Spread"].astype(str)
                                         .str.replace(r"[^\d.\-]", "", regex=True)
                                         .replace("", np.nan).astype(float))
    for col in ["S&P_Numeric", "Moody's_Numeric", "Fitch_Numeric"]:
        if col in df.columns:
            df[col] = (df[col].astype(str)
                                   .str.replace(r"[^\d.\-]", "", regex=True)
                                   .replace("", np.nan).astype(float))

    # ---------- scatter & ties --------------------------------------------
    fig, ax = plt.subplots()
    per_ctry = {}

    all_x_vals = []
    all_y_vals = []

    for colour, (col, lbl) in zip(
            palette,
            [("S&P_Numeric","S&P"),("Moody's_Numeric","Moody's"),
             ("Fitch_Numeric","Fitch")]):

        if col not in df.columns:
            continue
        sub = df.dropna(subset=[col, "Spread_Numeric"])

        jitter_x_vals = sub["Spread_Numeric"] + rng.normal(0, jitter_x, len(sub))
        y_vals        = sub[col]

        ax.scatter(jitter_x_vals, y_vals, s=140, c=[colour],
                   alpha=.85, label=lbl)

        all_x_vals.extend(sub["Spread_Numeric"].tolist())
        all_y_vals.extend(y_vals.tolist())

        for ctry, x0, y0 in zip(sub["Country"], sub["Spread_Numeric"], sub[col]):
            d = per_ctry.setdefault(ctry, {"x":x0, "ys":[]})
            d["ys"].append(y0)

    # vertical tie lines
    for d in per_ctry.values():
        if len(d["ys"]) > 1:
            ax.plot([d["x"], d["x"]], [min(d["ys"]), max(d["ys"])],
                    color="grey", lw=2, alpha=.55, zorder=2)

    # ---------- correlation line ------------------------------------------
    x_arr = np.array(all_x_vals)
    y_arr = np.array(all_y_vals)
    # fit linear regression
    m, b = np.polyfit(x_arr, y_arr, 1)
    xs = np.linspace(x_arr.min(), x_arr.max(), 100)
    ax.plot(xs, m*xs + b, color='black', lw=2, linestyle='--', zorder=1)
    # compute correlation coefficient
    corr = np.corrcoef(x_arr, y_arr)[0, 1]
    ax.text(0.95, 0.05, f"r = {corr:.2f}", transform=ax.transAxes,
            ha='right', va='bottom', fontsize=20, backgroundcolor='white', alpha=0.7)

    # ---------- flag or code labels ---------------------------------------
    for _, row in df.iterrows():
        ctry = row["Country"]
        code = COUNTRY_TO_CODE.get(ctry, ctry[:2].lower())
        x    = row["Spread_Numeric"]
        y    = np.nanmax([row.get(c) for c in
                         ["S&P_Numeric","Moody's_Numeric","Fitch_Numeric"]])

        lx, ly = x + label_dx, y
        icon   = load_icon(code)

        if icon is not None:
            zoom = flag_height / icon.shape[0]
            ab = AnnotationBbox(OffsetImage(icon, zoom=zoom),
                                (lx, ly), frameon=False,
                                box_alignment=(0,.5), zorder=4)
            ax.add_artist(ab)
        else:
            ax.text(lx, ly, code.upper(), fontsize=14, fontweight="bold",
                    va="center", zorder=4)

    # ---------- shaded background -----------------------------------------
    ax.axhspan(13, 22, color="#cce6ff", alpha=.25, zorder=0)   # investment
    ax.axhspan( 1, 13, color="#e8d5ff", alpha=.25, zorder=0)   # speculative
    ax.text(ax.get_xlim()[1]*.985, 13.2, "Investment Grade",
            ha="right", va="bottom", color="navy", fontsize=18, alpha=.8)
    ax.text(ax.get_xlim()[1]*.985, 12.8, "Speculative Grade",
            ha="right", va="top", color="#5e3b7f", fontsize=18, alpha=.8)

    # ---------- Y-axis letters --------------------------------------------
    ticks = range(1,23)
    ax.set_yticks(ticks)
    ax.set_yticklabels([RATING_LABEL[t] for t in ticks])

    # ---------- cosmetics --------------------------------------------------
    ax.set_xlabel("10-Year Bond Spread to US (bp)")
    ax.set_ylabel("Credit Rating")
    ax.set_title("Bond Spreads vs. Sovereign Credit Ratings", pad=45, fontweight='bold')

    pad = (x_arr.max() - x_arr.min()) * .05
    ax.set_xlim(x_arr.min()-pad, x_arr.max()+pad)
    # start y-axis at zero equivalent to show D grades clearly
    ax.set_ylim(0, 23.5)

    ax.axvline(0, color="black", lw=2, alpha=.4)
    ax.grid(alpha=.3)
    ax.legend()
    fig.tight_layout()
    fig.savefig(outfile, dpi=300, bbox_inches="tight")
    print("✅  Saved →", outfile)

# ---------------------------------------------------------------- run ---------
if __name__ == "__main__":
    if not CSV_FILE.exists():
        sys.exit("❌  credit_ratings_and_spreads.csv not found.")
    create_plot()


/var/folders/37/4hl523pd7rs6vzvf8bkrgz580000gn/T/ipykernel_89145/4132776333.py:144: RuntimeWarning: All-NaN axis encountered
  y    = np.nanmax([row.get(c) for c in


✅  Saved → bond_spreads_vs_ratings.png


In [43]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Investment‑grade bond spreads vs. sovereign credit ratings - FINAL VERSION

• BBB (14) to AAA only — BBB is the lowest rating shown
• Square 12 × 12‑inch figure, doubled fonts (same look as full plot)
• Vertical x = 0 reference line, investment‑grade shading
• Vertical ties between multiple agency ratings per country
• PNG flags (or ISO fallback) as labels with vertical jitter
• Dashed correlation line with Pearson r in lower‑right
• X‑axis fixed at ±500 bp
• Exactly matching original display with no white space issues
"""

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from pathlib import Path
import sys

# ---------------------------------------------------------------- paths -------
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:  # Jupyter or interactive
    BASE_DIR = Path.cwd()

CSV_FILE = BASE_DIR / "credit_ratings_and_spreads.csv"
ICON_DIR = BASE_DIR / "flag_icons"

# ---------------------------------------------------------------- ISO map ----
COUNTRY_TO_CODE = {
    "Switzerland": "ch", "Singapore": "sg", "Norway": "no", "Netherlands": "nl",
    "Germany": "de", "Australia": "au", "Sweden": "se", "Denmark": "dk",
    "Canada": "ca", "New Zealand": "nz", "United States": "us", "USA": "us",
    "Finland": "fi", "Austria": "at", "Qatar": "qa", "Taiwan": "tw",
    "Ireland": "ie", "South Korea": "kr", "Korea, South": "kr", "Hong Kong": "hk",
    "United Kingdom": "gb", "UK": "gb", "Belgium": "be", "Czech Republic": "cz",
    "Czechia": "cz", "France": "fr", "Iceland": "is", "Slovenia": "si",
    "Japan": "jp", "China": "cn", "Lithuania": "lt", "Malta": "mt",
    "Chile": "cl", "Portugal": "pt", "Slovakia": "sk", "Poland": "pl",
    "Spain": "es", "Croatia": "hr", "Cyprus": "cy", "Israel": "il",
    "Malaysia": "my", "Botswana": "bw", "Bulgaria": "bg", "Philippines": "ph",
    "Italy": "it", "Indonesia": "id", "Peru": "pe", "Kazakhstan": "kz",
    "Mexico": "mx", "Hungary": "hu", "Greece": "gr", "India": "in",
    "Mauritius": "mu", "Romania": "ro", "Colombia": "co", "Serbia": "rs",
    "Morocco": "ma", "Vietnam": "vn", "Brazil": "br", "South Africa": "za",
    "Jordan": "jo", "Namibia": "na", "Turkey": "tr", "Bangladesh": "bd",
    "Bahrain": "bh", "Uganda": "ug", "Nigeria": "ng", "Egypt": "eg",
    "Kenya": "ke", "Pakistan": "pk", "Sri Lanka": "lk", "Zambia": "zm",
    "Ukraine": "ua", "Russia": "ru"
}

# rating number → letter
RATING_LABEL = {
    14: "BBB", 15: "BBB+", 16: "A-", 17: "A", 18: "A+",
    19: "AA-", 20: "AA", 21: "AA+", 22: "AAA"
}

# ---------------------------------------------------------------- helpers ----
def load_icon(code: str):
    png = ICON_DIR / f"{code}.png"
    return plt.imread(png) if png.exists() else None

# ---------------------------------------------------------------- plot --------
def create_investment_plot(outfile="investment_grade_plot.png",
                           jitter_x=5, label_dx=8, flag_height=24):
    """Create perfect square plot matching original dimensions exactly: 3732 × 3566 pixels"""
    
    # Calculate figure size to get exact pixel dimensions at 300 dpi
    # Original dimensions: 3732 × 3566 pixels at 300 dpi
    # Figure size in inches = pixels / dpi
    width_inches = 3732 / 300
    height_inches = 3566 / 300
    
    # Reset any previous matplotlib settings
    mpl.rcdefaults()
    
    # Set global font settings
    plt.rcParams.update({
        'font.family': ["Arial", "Helvetica", "DejaVu Sans", "sans-serif"],
        'font.size': 12,
        'axes.titlesize': 40, 
        'axes.labelsize': 32,
        'xtick.labelsize': 24, 
        'ytick.labelsize': 24,
        'legend.fontsize': 24,
    })
    
    # Use fixed seed for reproducibility
    rng = np.random.default_rng(42)
    palette = sns.color_palette("bright", 3)
    
    # ---------- data -------------------------------------------------------
    df = pd.read_csv(CSV_FILE)
    df["Spread_Numeric"] = (df["Spread"].astype(str)
                             .str.replace(r"[^\d.\-]", "", regex=True)
                             .replace("", np.nan).astype(float))

    agency_cols = []
    for col in ["S&P_Numeric", "Moody's_Numeric", "Fitch_Numeric"]:
        if col in df.columns:
            df[col] = (df[col].astype(str)
                       .str.replace(r"[^\d.\-]", "", regex=True)
                       .replace("", np.nan).astype(float))
            agency_cols.append(col)

    # Keep only BBB (14) and higher
    df = df[df[agency_cols].max(axis=1) >= 14]
    
    # ---------- figure with precise dimensions -----------------------------
    # Use exact dimensions to match original at 300 dpi
    fig = plt.figure(figsize=(width_inches, height_inches))
    
    # Use exact positioning for the plot area - key to prevent whitespace issues
    ax = fig.add_axes([0.12, 0.12, 0.78, 0.75])  # left, bottom, width, height
    
    # ---------- scatter & ties --------------------------------------------
    per_ctry = {}
    all_x, all_y = [], []

    # Plot each agency's ratings
    for colour, (col, lbl) in zip(
            palette,
            [(c, c.split('_')[0]) for c in agency_cols]):
        
        sub = df.dropna(subset=[col, "Spread_Numeric"])
        jittered = sub["Spread_Numeric"] + rng.normal(0, jitter_x, len(sub))
        y_vals = sub[col]
        
        ax.scatter(jittered, y_vals, s=140, c=[colour], alpha=.85, label=lbl)
        
        all_x.extend(sub["Spread_Numeric"].tolist())
        all_y.extend(y_vals.tolist())
        
        for ctry, x0, y0 in zip(sub["Country"], sub["Spread_Numeric"], sub[col]):
            d = per_ctry.setdefault(ctry, {"x": x0, "ys": []})
            d["ys"].append(y0)

    # Vertical tie lines between multiple agency ratings for same country
    for d in per_ctry.values():
        if len(d["ys"]) > 1:
            ax.plot([d["x"], d["x"]], [min(d["ys"]), max(d["ys"])],
                    color="grey", lw=2, alpha=.55, zorder=2)

    # ---------- correlation line ------------------------------------------
    x_arr, y_arr = np.array(all_x), np.array(all_y)
    m, b = np.polyfit(x_arr, y_arr, 1)
    xs = np.linspace(-500, 500, 300)
    ax.plot(xs, m*xs + b, linestyle="--", lw=2, color="black", zorder=1)
    r = np.corrcoef(x_arr, y_arr)[0, 1]
    
    # Correlation text properly positioned
    ax.text(0.95, 0.05, f"r = {r:.2f}", transform=ax.transAxes,
            ha="right", va="bottom", fontsize=20, 
            bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.2'),
            zorder=5)

    # ---------- flag or code labels with vertical jitter ------------------
    used_positions = {}  # Track flag positions to avoid overlap
    
    for _, row in df.iterrows():
        ctry = row["Country"]
        code = COUNTRY_TO_CODE.get(ctry, ctry[:2].lower())
        x = row["Spread_Numeric"]
        y = np.nanmax([row.get(c, np.nan) for c in agency_cols])
        
        # Skip if invalid data
        if np.isnan(y) or np.isnan(x):
            continue
        
        # Calculate base position
        lx = x + label_dx
        ly = y
        
        # Apply vertical jitter based on nearby flags
        key_x = round(x / 20) * 20  # Round to identify nearby flags
        nearby_count = 0
        
        for test_y in range(int(y) - 1, int(y) + 2):
            if (key_x, test_y) in used_positions:
                nearby_count += used_positions[(key_x, test_y)]
        
        # Apply smart jitter to avoid overlaps
        if nearby_count > 0:
            jitter_direction = 1 if nearby_count % 2 == 1 else -1
            jitter_amount = (nearby_count // 2 + 1) * 0.3
            ly += jitter_direction * jitter_amount
        
        # Record this position
        used_positions[(key_x, int(y))] = used_positions.get((key_x, int(y)), 0) + 1
        
        # Add flag or ISO code
        icon = load_icon(code)
        if icon is not None:
            zoom = flag_height / icon.shape[0]
            ab = AnnotationBbox(OffsetImage(icon, zoom=zoom), (lx, ly),
                               frameon=False, box_alignment=(0, 0.5), zorder=4)
            ax.add_artist(ab)
        else:
            ax.text(lx, ly, code.upper(), fontsize=14, fontweight="bold", 
                   va="center", zorder=4)

    # ---------- shaded background -----------------------------------------
    ax.axhspan(14, 22, color="#cce6ff", alpha=.25, zorder=0)
    
    # Investment Grade label properly positioned
    ax.text(460, 14.2, "Investment Grade", ha="right", va="bottom",
           color="navy", fontsize=18, alpha=.8, zorder=5)

    # ---------- Y-axis letters --------------------------------------------
    ax.set_yticks(list(RATING_LABEL.keys()))
    ax.set_yticklabels(list(RATING_LABEL.values()))

    # ---------- cosmetics -------------------------------------------------
    ax.set_xlabel("10-Year Bond Spread to US (bp)")
    ax.set_ylabel("Credit Rating")
    
    # Place title directly on figure with precise position control
    fig.text(0.5, 0.94, "Bond Spreads vs. Sovereign Credit Ratings", 
            ha='center', fontsize=40, fontweight='bold')
    fig.text(0.5, 0.89, "Investment grade", 
            ha='center', fontsize=32)

    # Set exact axes limits
    ax.set_xlim(-500, 500)
    ax.set_ylim(13.8, 22.5)

    # Reference line at x=0
    ax.axvline(0, color="black", lw=2, alpha=.4)
    ax.grid(alpha=.3)
    
    # Legend in upper right, properly positioned
    ax.legend(loc='upper right', framealpha=0.9)
    
    # Ensure truly square data area
    ax.set_aspect('auto')
    
    # Save with exact dimensions
    fig.savefig(outfile, dpi=300, facecolor='white', edgecolor='none')
    
    # Verify the output file dimensions
    try:
        from PIL import Image
        img = Image.open(outfile)
        print(f"✅  Saved → {outfile} with dimensions {img.size[0]} × {img.size[1]} pixels")
    except ImportError:
        print(f"✅  Saved → {outfile}")
        print(f"    Expected dimensions: 3732 × 3566 pixels")
        print(f"    (Install PIL/Pillow to verify image dimensions)")


# ---------------------------------------------------------------- run ---------
if __name__ == "__main__":
    if not CSV_FILE.exists():
        sys.exit("❌  credit_ratings_and_spreads.csv not found.")
    create_investment_plot()

✅  Saved → investment_grade_plot.png with dimensions 3732 × 3566 pixels
